In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data_landsat()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_h,wp_le,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,2.605590,13.643902,0.377924,0.413754,0.594708,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,3.459300,13.498761,0.381184,0.420529,0.597960,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,2.117474,14.987823,0.384444,0.427304,0.601212,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,4.636145,11.355851,0.387704,0.434079,0.604465,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,3.448946,10.943102,0.390963,0.440854,0.607717,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 0 to 1027
Data columns (total 24 columns):
PET             1028 non-null float64
VPD             1028 non-null float64
air_temp        1028 non-null float64
doy             1028 non-null int64
precip          1028 non-null float64
soil_temp       1028 non-null float64
sw_in           1028 non-null float64
wind_speed      1028 non-null float64
year            1028 non-null int64
wp_RNET         1028 non-null float64
wp_ch4_gf       1028 non-null float64
wp_co2_gf       1028 non-null float64
wp_er           1028 non-null float64
wp_gpp          1028 non-null float64
wp_h            1028 non-null float64
wp_le           1028 non-null float64
wp_evi          1028 non-null float64
wp_lswi2        1028 non-null float64
wp_ndvi         1028 non-null float64
wp_LST.day      1028 non-null float64
wp_LST.night    1028 non-null float64
LW_IN.wp        1028 non-null float64
LW_IN.si        1028 non-null float64
LW_IN           1028 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_gpp"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1028, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.986666302388
CV Scores:  0.987479949286, 0.985162696564, 0.990130992366, 0.987578986555, 0.987117538399, 0.986975923496, 0.982577054666, 0.982955562857, 0.987573598292, 0.989110721399
OOB score: 0.987108011746
Feature Importances:
('wp_LST.night', 0.25076081191660704)
('wp_evi', 0.18982626110007506)
('wp_lswi2', 0.15111311820006607)
('wp_ndvi', 0.14466862686297532)
('air_temp', 0.082413369135439188)
('wp_LST.day', 0.060436908121492196)
('PET', 0.050945492332231543)
('sw_in', 0.035194204926820828)
('VPD', 0.019147530671230369)
('wind_speed', 0.0088925508991627676)
('LW_IN.wp', 0.0062692894604389924)
('precip', 0.00033183637346069522)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.986047136809
CV Scores:  0.984269022019, 0.98780260291, 0.98683449314, 0.983617981267, 0.989421791457, 0.988148693918, 0.985514339043, 0.978719880864, 0.988318698268, 0.987823865205
Feature Importances:
('wp_evi', 0.14566964806171442)
('wp_lswi2', 0.1367454895398342)
('wp_ndvi', 0.121628101961092)
('wp_LST.night', 0.10949715987976401)
('sw_in', 0.093565128957605465)
('wp_LST.day', 0.077694212538882784)
('VPD', 0.074588246536409442)
('LW_IN.wp', 0.065419719483959013)
('air_temp', 0.05845974923088984)
('wind_speed', 0.058227718898992614)
('PET', 0.055493132306970408)
('precip', 0.0030116926038856657)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.935494957287
CV Scores:  0.915463260607, 0.942813404107, 0.943441468266, 0.944390864714, 0.942105178182, 0.938584032745, 0.921472913967, 0.931250340564, 0.928207433222, 0.947220676495


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 3263.90210
Step #501, epoch #50, avg. loss: 107.18152
Step #1001, epoch #100, avg. loss: 16.77762
Step #1501, epoch #150, avg. loss: 11.72059
Step #2001, epoch #200, avg. loss: 8.74815
Step #2501, epoch #250, avg. loss: 7.09845
Step #3001, epoch #300, avg. loss: 5.88620
Step #3501, epoch #350, avg. loss: 4.94531
Step #4001, epoch #400, avg. loss: 4.25517
Step #4501, epoch #450, avg. loss: 3.74860
Step #1, avg. loss: 2312.33936
Step #501, epoch #50, avg. loss: 109.57610
Step #1001, epoch #100, avg. loss: 15.89971
Step #1501, epoch #150, avg. loss: 10.52064
Step #2001, epoch #200, avg. loss: 8.30281
Step #2501, epoch #250, avg. loss: 6.54048
Step #3001, epoch #300, avg. loss: 5.65676
Step #3501, epoch #350, avg. loss: 4.74202
Step #4001, epoch #400, avg. loss: 4.18825
Step #4501, epoch #450, avg. loss: 3.70185
Step #1, avg. loss: 2900.85718
Step #501, epoch #50, avg. loss: 109.02621
Step #1001, epoch #100, avg. loss: 15.73230

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x10ad5a488>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)